# Security Scripts - Amendments  
                       ![image](https://user-images.githubusercontent.com/30621653/75694124-ecc35080-5c9f-11ea-9335-75a04f32b64b.png)


## Contents
> **Logins**
> - Create logins (Query 1)
 - Find orphaned users (Query 2)
- Fix Orphaned users  (Query 3)

> **Passwords**
- Update user passwords (Query 4)
- Add user to sysadmin - AD Connection (Query 5)
- Add use to server and sysadmin SQL user (Query 6)
- Add User to database role - data reader (query 7)

### <U>**Security Adding / Updating permissions (Query 1)</U>**

>CREATE LOGIN login_name { WITH [option_list1] | FROM {sources} }   
    [option_list1] ::=  
    PASSWORD = { 'password' | hashed_password HASHED } [ MUST_CHANGE ]  
    [ , {option_list2} [ ,... ] ]  

>{option_list2} ::=  
    SID = sid  
    | DEFAULT_DATABASE = database  
    | DEFAULT_LANGUAGE = language  
    | CHECK_EXPIRATION = { ON | OFF}  
    | CHECK_POLICY = { ON | OFF}  
    | CREDENTIAL = credential_name  

>{sources} ::= 
    WINDOWS [ WITH <windows_options>[ ,... ] ]  
    | CERTIFICATE certname  
    | ASYMMETRIC KEY asym_key_name  

>{windows_options} ::=  
    DEFAULT_DATABASE = database  
    | DEFAULT_LANGUAGE = language  

In [ ]:
CREATE LOGIN DecommissionRO WITH PASSWORD = '1hgcvkbEXzEKKz6LA0KU' 
GO

### **<u>Orphaned users Query(2) </u>**
> Show current ophaned users in current DB

In [ ]:
EXEC sp_change_users_login 'report'

### <u>**Fixing Change users login (Query 3)**</u>

>sp_change_users_login [ @Action = ] 'action' 
 [ , [ @UserNamePattern = ] 'user' ] 
 [ , [ @LoginName = ] 'login' ] 
 [ , [ @Password = ] 'password' ] 
[;] 


>**Auto_Fix**	Links a user entry in the sys.database_principals system catalog view in the current database to a SQL Server login of the same name. If a login with the same name does not exist, one will be created. Examine the result from the Auto_Fix statement to confirm that the correct link is in fact made. Avoid using Auto_Fix in security-sensitive situations.
When you use Auto_Fix, you must specify user and password if the login does not already exist, otherwise you must specify user but password will be ignored. login must be NULL. user must be a valid user in the current database. The login cannot have another user mapped to it. 

>**Report**	Lists the users and corresponding security identifiers (SID) in the current database that are not linked to any login. user, login, and password must be NULL or not specified.
,To replace the report option with a query using the system tables, compare the entries in sys.server_prinicpals with the entries in sys.database_principals. 
 
>**Update_One**	Links the specified user in the current database to an existing SQL Server login. user and login must be specified. password must be NULL or not specified.

In [ ]:
EXEC sp_change_users_login 'Update_one','DecommissionRO','DecommissionRO'

### **Update user password (Query 4)**

>ALTER LOGIN login_name    
 {  
  {status_option}  
    | WITH {set_option} [ ,... ]    
    | {cryptographic_credential_option}    
 }    
[;]   

>{status_option} ::=  
    ENABLE | DISABLE  

>{set_option} ::=  
    PASSWORD = 'password' | hashed_password HASHED  
 [
  OLD_PASSWORD = 'oldpassword' | {password_option} [<password_option> ]  
 ]
     | DEFAULT_DATABASE = database  
     | DEFAULT_LANGUAGE = language  
     | NAME = login_name  
     | CHECK_POLICY = { ON | OFF }  
     | CHECK_EXPIRATION = { ON | OFF }  
     | CREDENTIAL = credential_name  
     | NO CREDENTIAL  

>{password_option} ::=  
  MUST_CHANGE | UNLOCK  

>{cryptographic_credentials_option} ::=  
  ADD CREDENTIAL credential_name  
     | DROP CREDENTIAL credential_name  

In [ ]:
USE [master]
GO
ALTER LOGIN [username] WITH PASSWORD=N'password'
GO

### <u>**Add user to sysadmin - AD Connection (Query 5)**</U>
 
>ALTER SERVER ROLE server_role_name   
{  
    [ ADD MEMBER server_principal ]  
    | [ DROP MEMBER server_principal ]  
    | [ WITH NAME = new_server_role_name ]  
} [ ; ]

In [ ]:
USE [master]
GO
ALTER SERVER ROLE [serveradmin] ADD MEMBER [ADGROUP]
GO


### <u>**Add use to server and sysadmin SQL user (Query 6)**</u>

> sp_addsrvrolemember [ @loginame= ] 'login'   
    , [ @rolename = ] 'role'  


This feature is in maintenance mode and may be removed in a future version of Microsoft SQL Server. Avoid using this feature in new development work, and plan to modify applications that currently use this feature. Use ALTER SERVER ROLE instead.

In [ ]:
USE [master]
GO
CREATE LOGIN [testuser] WITH PASSWORD=N'SecurePassword', DEFAULT_DATABASE=[master], CHECK_EXPIRATION=OFF, CHECK_POLICY=OFF
GO

In [ ]:
EXEC master..sp_addsrvrolemember @loginame = N'DOMAIN\USERNAME', @rolename = N'sysadmin'
GO



>ALTER SERVER ROLE server_role_name  
{ 
    [ ADD MEMBER server_principal ]  |  
    [ DROP MEMBER server_principal ] |  
    [ WITH NAME = new_server_role_name ]   
} [ ; ] 

In [ ]:

ALTER SERVER ROLE [testuser] ADD MEMBER [DOMAIN\USER]
GO


## <U>**Add User to database role - data reader (query 7)** </U>

>ALTER ROLE  role_name     
{    
    ADD MEMBER database_principal    
    |  DROP MEMBER database_principal    
    |  WITH NAME = new_name    
}    
[;]   

> db_datareader, db_datawriter, db_accessadmin, db_backupoperator, db_denydatareader, db_denydatawriter,db_owner, db_securityadmin

In [ ]:
USE [DatabaseName]

ALTER ROLE db_datareader ADD MEMBER [DOMAIN\USERNAME]

# [Next AG groups](file:///C:/sqlpresentations/2020/SQLBelgium/02_AvailabilityGroups.ipynb)